In [ ]:
# Load the YOLOv10 model
model = YOLO("yolov10n.pt")  # Load the appropriate YOLOv10 model

# Example: Train the model using the loaded dataset
def train_model(model, train_loader, val_loader, epochs=10, initial_lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    criterion = nn.CrossEntropyLoss()  # Adjust as per YOLOv10 (YOLO models typically use different loss functions)

    # To store learning rate history for plotting
    lr_history = []

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()

            # Forward pass (Make sure the model is expecting images in the correct format)
            outputs = model(inputs)

            # Compute loss (adjust loss computation based on your model)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Step the scheduler
        scheduler.step()

        # Store the current learning rate for plotting
        lr_history.append(optimizer.param_groups[0]['lr'])

        print(f'Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader):.4f}')
    
    # Plot learning rate graph
    plt.plot(range(epochs), lr_history, label='Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.legend()
    plt.show()

# Train the model
train_model(model, train_loader, val_loader, epochs=10, initial_lr=0.001)


In [ ]:
from ultralytics import YOLO

# Load the YOLOv10 model
model = YOLO("yolov10n.pt")  # Ensure this is the correct model file

# Path to the custom dataset YAML file
yaml_file = "/home/dharun/Desktop/pest_detection/Pest_dectection/Custom Dataset After Augmentation/data.yaml"

# Start training using the custom dataset
model.train(data=yaml_file, epochs=3, batch=32, imgsz=640)


In [ ]:
#to check latency
from ultralytics import YOLO
import time
import cv2
import matplotlib.pyplot as plt

# Load a COCO-pretrained YOLOv5 model and force it to run on the CPU
model = YOLO("/home/dharun/Desktop/pest_detection/yolov8.pt")
model.to("cpu")  # Ensure model runs on CPU

# Open the video file
video_path = '/home/dharun/Desktop/pest_detection/videoplayback.mp4'  # Replace with your video path
cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print("Error: Couldn't open video.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))

# List to store latencies
latencies = []
frame_numbers = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break  # End of video

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Get current frame number

    # Measure latency for each frame
    start_time = time.time()  # Start timer

    # Run inference on the frame (model inference on CPU)
    results = model(frame)  # Inference on the current frame
    
    end_time = time.time()  # End timer
    
    # Calculate latency (in seconds)
    latency = end_time - start_time

    # Append latency and frame number for graphing
    latencies.append(latency)
    frame_numbers.append(frame_number)

    # Draw results on the frame
    annotated_frame = results[0].plot()  # Annotate frame with detection results

    # Write the annotated frame to output video
    out.write(annotated_frame)

    # Optional: Show the annotated frame (for testing purposes)
    # cv2.imshow("Frame", annotated_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Plot the latency graph
plt.figure(figsize=(10, 6))
plt.plot(frame_numbers, latencies, label="Inference Latency", color='b', marker='o', linestyle='-', markersize=4)
plt.title("Inference Latency per Frame (CPU)")
plt.xlabel("Frame Number")
plt.ylabel("Latency (seconds)")

# Dynamically adjust the y-axis based on the observed latencies
min_latency = min(latencies)
max_latency = max(latencies)

# Add a margin of 10% around the min/max latency for better visualization
margin = (max_latency - min_latency) * 0.1
plt.ylim(min_latency - margin, max_latency + margin)

plt.grid(True)

# Add grid and legend
plt.legend()
plt.tight_layout()
plt.show()
